# Test BSM model and Normal model implementation

In [16]:
import numpy as np
from option_models import bsm
from option_models import normal

In [17]:
### only run this when you changed the class definition
import imp
imp.reload(bsm)
imp.reload(normal)

<module 'option_models.normal' from 'G:\\PHBS\\Second Year\\Semester1\\Module1\\ASP\\Github\\PHBS_ASP_2018\\HW3\\option_models\\normal.py'>

## BSM model

### Price
$$ Payoff=I_{sign}e^{-rT}[F\,N(I_{sign}d_1)-K\,N(I_{sign}d_2)] \\
d_1 = \frac{log(F/K)}{\sigma\sqrt{T}}+\frac{1}{2}\sigma\sqrt{T}\\
d_2 = d_1-\sigma\sqrt{T} \\
F = S_0e^{(r-q)T}
$$
$$
I_{sign}=1,\;means\; call\;option
$$

In [8]:
# create model
bsm1 = bsm.BsmModel(0.2)

In [9]:
# price
strike = 102
spot = 100
texp = 0.25

price = bsm1.price(strike=strike, spot=spot, texp=texp, cp_sign=1)
print(price)
assert( abs(price - 3.10628366655) < 1e-10 )

3.1062836665495652


### Implied vol

In [6]:
# Randomly generate spot/strike/expiry/intr/divr/cp_sign
# Then test implied volatility

for k in range(100):
    spot = np.random.uniform(80,100)
    strike = np.random.uniform(80,100)
    vol = np.random.uniform(0.0001, 0.4)
    texp = np.random.uniform(0.1, 5)
    intr = np.random.uniform(0, 0.3)
    divr = np.random.uniform(0, 0.3)
    cp_sign = 1 if np.random.rand() > 0.5 else -1

    #print( spot, strike, vol, texp, intr, divr, cp_sign)

    bsm2 = bsm.BsmModel(vol=vol, intr=intr, divr=divr)
    price = bsm2.price(strike, spot, texp, cp_sign )
    
    # get implied vol
    vol_imp = bsm2.impvol(price, strike, spot, texp=texp, cp_sign=cp_sign)
    
    # now price option with the obtained implied vol
    bsm2.vol = vol_imp
    price_imp = bsm2.price(strike, spot, texp, cp_sign )
    
    # compare the two prices
    assert( abs(price - price_imp) < 1e-8 )

### Delta
Verify delta by comparing numerical derivative
$$
\Delta = I_{sign}e^{-qT}N(I_{sign}d_1)
$$

In [7]:
delta = bsm1.delta(strike=strike, spot=spot, texp=texp, cp_sign=1)
h = 1e-6
price_up = bsm1.price(strike=strike, spot=spot+h, texp=texp, cp_sign=1)
price_dn = bsm1.price(strike=strike, spot=spot-h, texp=texp, cp_sign=1)
delta_numeric = ( price_up - price_dn )/(2*h)
assert( abs(delta - delta_numeric) < 1e-6 )

### Vega
$$
Vega = S_0e^{-qT}n(d_1)\sqrt{T} = Ke^{-rT}n(d_2)\sqrt{T}
$$

In [12]:
vol = 0.2
bsm1.vol = vol
vega = bsm1.vega(strike=strike, spot=spot, texp=texp, cp_sign=1)
v = 1e-6

bsm1.vol = vol+v
price_up = bsm1.price(strike=strike, spot=spot, texp=texp, cp_sign=1)
bsm1.vol = vol-v
price_dn = bsm1.price(strike=strike, spot=spot, texp=texp, cp_sign=1)
vega_numeric = ( price_up - price_dn )/(2*v)
assert( abs(vega - vega_numeric) < 1e-6 )

In [13]:
vega,vega_numeric

(65.30903667898743, 65.30903668533483)

### Gamma
$$
\Gamma = e^{-qT}\frac{n(d_1)}{S_0\sigma\sqrt{T}} = Ke^{-rT}\frac{n(d_2)}{S_0^2\sigma\sqrt{T}}
$$

In [19]:
bsm1.vol = vol
gamma = bsm1.gamma(strike=strike, spot=spot, texp=texp, cp_sign=1)
h = 1e-6
delta_up = bsm1.delta(strike=strike, spot=spot+h, texp=texp, cp_sign=1)
delta_dn = bsm1.delta(strike=strike, spot=spot-h, texp=texp, cp_sign=1)
gamma_numeric = ( delta_up - delta_dn )/(2*h)
assert( abs(gamma - gamma_numeric) < 1e-6 )

In [20]:
gamma,gamma_numeric

(0.0121781729008249, 0.01217817285903422)

## Normal Model

In [6]:
# create model
normal1 = normal.NormalModel(20) # vol multiply by spot

### Price
* The price is diferent because the exact price of option may be priced by  BSM.
$$ \\
Payoff=e^{-rT}I_{sign}(F-K)N(I_{sign}d) +\sigma\sqrt{T}n(d)\\
d = \frac{F-K}{\sigma\sqrt{T}}\\
F = S_0e^{(r-q)T}
$$
$$
I_{sign}=1,\;means\; call\;option
$$

In [7]:
# price
strike = 102
spot = 100
texp = 0.25

price = normal1.price(strike=strike, spot=spot, texp=texp, cp_sign=1)
print(price)
assert( abs(price - 3.10628366655) < 1e-10 )

3.0689463586327648


AssertionError: 

### Implied Vol

In [8]:
# Randomly generate spot/strike/expiry/intr/divr/cp_sign
# Then test implied volatility

for k in range(100):
    spot = np.random.uniform(80,100)
    strike = np.random.uniform(80,100)
    vol = np.random.uniform(1, 50)
    texp = np.random.uniform(0.1, 5)
    intr = np.random.uniform(0, 0.3)
    divr = np.random.uniform(0, 0.3)
    cp_sign = 1 if np.random.rand() > 0.5 else -1

    #print( spot, strike, vol, texp, intr, divr, cp_sign)

    normal2 = normal.NormalModel(vol=vol, intr=intr, divr=divr)
    price = normal2.price(strike, spot, texp, cp_sign )
    
    # get implied vol
    vol_imp = normal2.impvol(price, strike, spot, texp=texp, cp_sign=cp_sign)
    
    # now price option with the obtained implied vol
    normal2.vol = vol_imp
    price_imp = normal2.price(strike, spot, texp, cp_sign )
    
    # compare the two prices
    assert( abs(price - price_imp) < 1e-8 )

### Delta
$$
\Delta = I_{sign}N(I_{sign}d)
$$

In [15]:
delta = normal1.delta(strike=strike, spot=spot, texp=texp, cp_sign=1)
h = 1e-6
price_up = normal1.price(strike=strike, spot=spot+h,texp=texp, cp_sign=1)
price_dn = normal1.price(strike=strike, spot=spot-h,texp=texp, cp_sign=1)
delta_numeric = ( price_up - price_dn )/(2*h)
assert( abs(delta - delta_numeric) < 1e-6 )
print(delta,delta_numeric)

0.5795466792951147 0.5795466773150793


### Vega
$$
Vega = \sqrt{T}n(d)
$$

In [14]:
vol = 20
normal1.vol = vol
vega = normal1.vega(strike=strike, spot=spot, texp=texp, cp_sign=1)
v = 1e-6

normal1.vol = vol+v
price_up = normal1.price(strike=strike, spot=spot, texp=texp, cp_sign=1)
normal1.vol = vol-v
price_dn = normal1.price(strike=strike, spot=spot, texp=texp, cp_sign=1)
vega_numeric = ( price_up - price_dn )/(2*v)
assert( abs(vega - vega_numeric) < 1e-6 )
print(vega,vega_numeric)

0.6285605563493409 0.62856055649263


### Gamma
$$
\Gamma = \frac{n(d)}{\sigma\sqrt{T}}
$$

In [13]:
normal1.vol = vol
gamma = normal1.gamma(strike=strike, spot=spot, texp=texp, cp_sign=1)
h = 1e-6
delta_up = normal1.delta(strike=strike, spot=spot+h, texp=texp, cp_sign=1)
delta_dn = normal1.delta(strike=strike, spot=spot-h, texp=texp, cp_sign=1)
gamma_numeric = ( delta_up - delta_dn )/(2*h)
assert( abs(gamma - gamma_numeric) < 1e-6 )
print(gamma,gamma_numeric)

0.012160278078823285 0.012160278062278707


In [17]:
x = 74.32/764
y = x
z = 40.01/206

In [23]:
34075*x*y*z,33670*x*y*z,33859*x*y*z

(62.627039223277016, 61.882682630894706, 62.230049040673116)

In [27]:
'小球理论体积值： ',4*np.pi*(5/2)**3/3

('小球理论体积值： ', 65.44984694978736)

In [28]:
'小球理论面积值： ',4*np.pi*(5/2)**2

('小球理论面积值： ', 78.53981633974483)

In [29]:
import numpy as np
import scipy.stats as ss
import scipy.optimize as sopt


In [18]:
np.log10(100/108),np.log(100/108)

(-0.0334237554869497, -0.07696104113612832)